In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, timezone
import requests
import os
from time import sleep
import pytz
from scipy.stats import pearsonr
from Functions import check_correlation, analyze_events, wrap_analyze_events, load_data, data_preparation, wrap_analyze_events_multiprocessing, show_output, round_
import pandas as pd
#pd.set_option('display.max_rows', None)
from time import time
from multiprocessing import Process
from multiprocessing import Lock, Pool, Manager
from time import time

/home/alberto/Docker/Trading/analysis/Functions.py:182: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if key is not 'coins' or key is not 'events' and len(shared_data[key]['coins']) > 0:
/home/alberto/Docker/Trading/analysis/Functions.py:182: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if key is not 'coins' or key is not 'events' and len(shared_data[key]['coins']) > 0:


In [2]:
start_interval=datetime(2023,5,7)
end_interval=datetime.now()

filter_position=(0,100)

data, df = load_data(start_interval=start_interval, end_interval=end_interval, filter_position=filter_position)
df

StatusCode for getting most_traded_coins: 200
new Json saved in /home/alberto/Docker/Trading/tracker/json/most_traded_coins.json
['/home/alberto/Docker/Trading/analysis/json/data-8-5-2023-122449.json']
Retrieving data from /home/alberto/Docker/Trading/analysis/json/data-8-5-2023-122449.json
92 coins have been loaded


,n_observations
AGIXUSDT,4775
DOGEUSDT,4775
ALGOUSDT,4775
MANAUSDT,4775
SHIBUSDT,4775
...,...
VETUSDT,4775
GALAUSDT,4775
XRPUSDT,4775
ADAUSDT,4775


In [ ]:
# WRAP ANALYSIS EVENTS
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.55, 0.60, 0.7, 0.8]
list_event_volume = [3,4,5,6]

resp = wrap_analyze_events(data, list_buy_vol, list_vol, list_minutes, list_event_buy_volume,
                    list_event_volume, start_interval, end_interval)


In [6]:
# WRAP ANALYSIS EVENTS
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m',]
list_minutes = [5, 15, 30]
list_event_buy_volume = [0.55, 0.60]
list_event_volume = [5,6]
resp = None
resp = wrap_analyze_events(data, list_buy_vol, list_vol, list_minutes, list_event_buy_volume,
                    list_event_volume, start_interval, end_interval)
df = pd.DataFrame(resp).transpose()
df

6 total combinations
25% Completed


/home/alberto/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


50% Completed
75% Completed
100% Completed
2.4129858016967773 seconds spent to run wrap_analyze_events


,mean,std,coins,events
buy_vol_5m:0.55/vol_5m:5/timeframe:5,-4.98,7.27,81.0,687.0
buy_vol_5m:0.55/vol_5m:6/timeframe:5,-4.96,7.72,76.0,518.0
buy_vol_5m:0.6/vol_5m:5/timeframe:5,-4.84,7.75,76.0,510.0
buy_vol_5m:0.6/vol_5m:6/timeframe:5,-4.83,8.26,69.0,373.0
buy_vol_5m:0.55/vol_5m:5/timeframe:15,-5.00,7.08,81.0,516.0
buy_vol_5m:0.55/vol_5m:6/timeframe:15,-4.98,7.53,76.0,401.0
buy_vol_5m:0.6/vol_5m:5/timeframe:15,-4.81,7.66,76.0,409.0
buy_vol_5m:0.6/vol_5m:6/timeframe:15,-4.84,8.12,69.0,309.0
buy_vol_5m:0.55/vol_5m:5/timeframe:30,-5.09,6.96,81.0,443.0
buy_vol_5m:0.55/vol_5m:6/timeframe:30,-5.06,7.31,76.0,343.0


In [3]:
# ANALYSIS MULTIPROCESSING 2
t1 = time()

data_arguments = data_preparation(data, n_slices = 6)

# MULTIPROCESSING WRAP ANALYSIS EVENTS
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m',]
list_minutes = [5, 15, 30]
list_event_buy_volume = [0.55, 0.60]
list_event_volume = [5,6]

total_combinations = len(list_buy_vol) * len(list_vol) * len(list_minutes) * len(list_event_volume) * len(list_event_buy_volume)
print('total_combinantions', ': ', total_combinations)
path = "/home/alberto/Docker/Trading/analysis/analysis_json/"



now = datetime.now()
now_year = str(now.year)
now_month = str(now.month)
now_day = str(now.day)
now_hour = str(now.hour)
now_minute = str(now.minute)
file_path = path + now_month+now_day+ '_' + now_hour+now_minute + '.json'

manager = Manager()
# Create shared memory for JSON data
shared_data = manager.Value(str, json.dumps({}))
lock = Manager().Lock()


# Create a multiprocessing Pool
pool = Pool()

# Execute the function in parallel
pool.starmap(wrap_analyze_events_multiprocessing, [(arg, arg_i, list_buy_vol, list_vol, list_minutes,
                                    list_event_buy_volume, list_event_volume, start_interval,
                                    end_interval, file_path, lock, shared_data) for arg, arg_i in zip(data_arguments, range(1,len(data_arguments)+1))])

# Close the pool
pool.close()
pool.join()
t2 = time()
print(t2-t1, ' seconds')


total_combinantions :  12


/home/alberto/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret

2.1541056632995605  seconds


In [4]:
shared_data.value

'{"buy_vol_5m:0.55/vol_5m:5/timeframe:5": {"mean": [-4.891100121802543, -4.791751852952848, -3.9750272367665875, -6.995471621317817, -3.66173665962256, -4.18084575500115, -8.166913059827422], "std": [4.3440526424374175, 6.615854378140579, 10.107987383555901, 2.84178796670995, 4.395645172771514, 9.176050372003564, 5.042139870173032], "coins": ["AGIXUSDT", "DOGEUSDT", "ALGOUSDT", "MANAUSDT", "SHIBUSDT", "STXUSDT", "DOTUSDT", "LINKUSDT", "APEUSDT", "JOEUSDT", "APTUSDT", "FLOKIUSDT", "AUDUSDT", "BETAUSDT", "KAVAUSDT", "ROSEUSDT", "ORNUSDT", "NEOUSDT", "CHZUSDT", "JASMYUSDT", "ARBUSDT", "CRVUSDT", "SOLUSDT", "SUIUSDT", "KLAYUSDT", "IMXUSDT", "RNDRUSDT", "HOOKUSDT", "ICPUSDT", "LTCUSDT", "TRXUSDT", "UMAUSDT", "FETUSDT", "AVAXUSDT", "SXPUSDT", "MASKUSDT", "SNXUSDT", "CAKEUSDT", "SANDUSDT", "ADAUSDT", "CFXUSDT", "EGLDUSDT", "ATOMUSDT", "MINAUSDT", "LINAUSDT", "EURUSDT", "ETCUSDT", "RLCUSDT", "FILUSDT", "GRTUSDT", "TOMOUSDT", "UNIUSDT", "ARPAUSDT", "RIFUSDT", "MATICUSDT", "WOOUSDT", "NEARUSDT",

In [5]:
output = show_output(shared_data)
df = pd.DataFrame(output).transpose()
df

,mean,std,n_coins,n_events
buy_vol_5m:0.55/vol_5m:5/timeframe:5,-5.104173,6.663567,81.0,687.0
buy_vol_5m:0.55/vol_5m:6/timeframe:5,-5.149643,7.131205,76.0,518.0
buy_vol_5m:0.6/vol_5m:5/timeframe:5,-4.967305,7.173038,76.0,510.0
buy_vol_5m:0.6/vol_5m:6/timeframe:5,-5.083340,7.719286,69.0,373.0
buy_vol_5m:0.55/vol_5m:5/timeframe:15,-5.085188,6.523288,81.0,516.0
buy_vol_5m:0.55/vol_5m:6/timeframe:15,-5.181692,6.924670,76.0,401.0
buy_vol_5m:0.6/vol_5m:5/timeframe:15,-4.925645,7.014487,76.0,409.0
buy_vol_5m:0.6/vol_5m:6/timeframe:15,-5.071807,7.524517,69.0,309.0
buy_vol_5m:0.55/vol_5m:5/timeframe:30,-5.140790,6.380423,81.0,443.0
buy_vol_5m:0.55/vol_5m:6/timeframe:30,-5.204486,6.750979,76.0,343.0


In [7]:
df = pd.DataFrame(resp).transpose()
df

,mean,std,coins,events
buy_vol_5m:0.55/vol_5m:5/timeframe:5,-4.98,7.27,81.0,687.0
buy_vol_5m:0.55/vol_5m:6/timeframe:5,-4.96,7.72,76.0,518.0
buy_vol_5m:0.6/vol_5m:5/timeframe:5,-4.84,7.75,76.0,510.0
buy_vol_5m:0.6/vol_5m:6/timeframe:5,-4.83,8.26,69.0,373.0
buy_vol_5m:0.55/vol_5m:5/timeframe:15,-5.00,7.08,81.0,516.0
buy_vol_5m:0.55/vol_5m:6/timeframe:15,-4.98,7.53,76.0,401.0
buy_vol_5m:0.6/vol_5m:5/timeframe:15,-4.81,7.66,76.0,409.0
buy_vol_5m:0.6/vol_5m:6/timeframe:15,-4.84,8.12,69.0,309.0
buy_vol_5m:0.55/vol_5m:5/timeframe:30,-5.09,6.96,81.0,443.0
buy_vol_5m:0.55/vol_5m:6/timeframe:30,-5.06,7.31,76.0,343.0
